# NYC Measles Cases by Neighborhood

* This Jupyter/Python notebook creates and saves map visualizations of the 2018-2019 NYC Measles Cases by Neighborhood
* This notebook is part of the "measles" GitHub project: https://github.com/carlos-afonso/measles
* Author: Carlos Afonso: https://carlos-afonso.github.io
* Date: June 3, 2019

## Import dependencies

In [1]:
from datetime import datetime
import folium
import imageio
import pandas as pd
import re

## Get measles data

### Read data from CSV file

In [2]:
measles_df = pd.read_csv('../data/nyc_health_measles_cases_by_neighborhood.csv')

measles_df

,Neighborhood,Ongoing transmission (2019-05-29),All cases (2018-09-01 to 2019-05-29),Newest cases (2019-05-24 to 2019-05-29)
0,Borough Park,1,85,3
1,Sunset Park,1,12,0
2,Williamsburg,1,424,5
3,Bensonhurst,0,2,0
4,Brighton Beach/Coney Island,0,4,0
5,Chelsea/Clinton,0,1,1
6,Crown Heights,0,3,0
7,Far Rockaway,0,1,0
8,Flatbush,0,1,0
9,Flushing,0,3,0


### Rename the data columns

In [3]:
original_cols = measles_df.columns

original_cols

Index(['Neighborhood', 'Ongoing transmission (2019-05-29)',
       'All cases (2018-09-01 to 2019-05-29)',
       'Newest cases (2019-05-24 to 2019-05-29)'],
      dtype='object')

In [4]:
new_cols = ['neighborhood', 'ongoing_transmission', 'all_cases', 'new_cases']

measles_df.columns = new_cols

measles_df

,neighborhood,ongoing_transmission,all_cases,new_cases
0,Borough Park,1,85,3
1,Sunset Park,1,12,0
2,Williamsburg,1,424,5
3,Bensonhurst,0,2,0
4,Brighton Beach/Coney Island,0,4,0
5,Chelsea/Clinton,0,1,1
6,Crown Heights,0,3,0
7,Far Rockaway,0,1,0
8,Flatbush,0,1,0
9,Flushing,0,3,0


### Extract total(s)

In [5]:
all_cases_total = measles_df[measles_df['neighborhood'] == 'TOTAL']['all_cases'].values[0]

new_cases_total = measles_df[measles_df['neighborhood'] == 'TOTAL']['new_cases'].values[0]

[all_cases_total, new_cases_total]

[550, 16]

### Extract dates

In [6]:
original_cols

Index(['Neighborhood', 'Ongoing transmission (2019-05-29)',
       'All cases (2018-09-01 to 2019-05-29)',
       'Newest cases (2019-05-24 to 2019-05-29)'],
      dtype='object')

In [7]:
[end_date_iso, all_start_date_iso, new_start_date_iso] = list(map(
    lambda x: re.search(r'\d{4}-\d{2}-\d{2}', x).group(), 
    original_cols[1:4]
))

[end_date, all_start_date, new_start_date] = list(map(
    lambda x: datetime.strptime(x, '%Y-%m-%d').date(), 
    [end_date_iso, all_start_date_iso, new_start_date_iso]
))

[end_date_label, all_start_date_label, new_start_date_label] = list(map(
    lambda x: x.strftime('%b %d, %Y').replace(' 0', ' '), 
    [end_date, all_start_date, new_start_date]
))

[
    ['All start date', all_start_date_iso, all_start_date, all_start_date_label], 
    ['New start date', new_start_date_iso, new_start_date, new_start_date_label], 
    ['End date      ', end_date_iso,       end_date,       end_date_label]
]

[['All start date', '2018-09-01', datetime.date(2018, 9, 1), 'Sep 1, 2018'],
 ['New start date', '2019-05-24', datetime.date(2019, 5, 24), 'May 24, 2019'],
 ['End date      ', '2019-05-29', datetime.date(2019, 5, 29), 'May 29, 2019']]

## Define locations data

In [8]:
locations_data = [
    #['Neighborhood', Latitude, Longitude]
    
    ['Bensonhurst',  40.6139, -73.9922], 
    ['Borough Park', 40.6350, -73.9921], 
    
    #['Brighton Beach', 40.5781, -73.9597], 
    #['Coney Island', 40.5755,  -73.9707], 
    ['Brighton Beach/Coney Island', (40.5781 + 40.5755)/2, -(73.9597 + 73.9707)/2], 
    
    # NOTE: "Clinton" is the same as "Hell's Kitchen"
    #['Chelsea', 40.7465, -74.0014], 
    #['Clinton', 40.7638, -73.9918],
    ['Chelsea/Clinton', (40.7465 + 40.7638)/2, -(74.0014 + 73.9918)/2], 
    
    ['Crown Heights', 40.6694, -73.9422], 
    ['Far Rockaway',  40.5999, -73.7448], 
    ['Flatbush',      40.6415, -73.9594], 
    ['Flushing',      40.7675, -73.8331], 
    
    #['Hunts Point', 40.8094, -73.8803], 
    #['Longwood',    40.8248, -73.8916], 
    #['Melrose',     40.8245, -73.9104], 
    ['Hunts Point/Longwood/Melrose', (40.8094 + 40.8248 + 40.8245)/3, -(73.8803 + 73.8916 + 73.9104)/3], 
    
    ['Jamaica', 40.7027, -73.7890], 
    
    #['Midwood',     40.6204, -73.9600], 
    #['Marine Park', 40.6114, -73.9332], 
    ['Midwood/Marine Park', (40.6204 + 40.6114)/2, -(73.9600 + 73.9332)/2], 
    
    ['Port Richmond', 40.6355, -74.1255], 
    ['Sunset Park',  40.6527, -74.0093], 
    ['Williamsburg', 40.7081, -73.9571], 
    ['Willowbrook', 40.6032, -74.1385]
]

locations_df = pd.DataFrame(locations_data, columns = ['neighborhood', 'latitude', 'longitude']) 

locations_df

,neighborhood,latitude,longitude
0,Bensonhurst,40.613900,-73.9922
1,Borough Park,40.635000,-73.9921
2,Brighton Beach/Coney Island,40.576800,-73.9652
3,Chelsea/Clinton,40.755150,-73.9966
4,Crown Heights,40.669400,-73.9422
5,Far Rockaway,40.599900,-73.7448
6,Flatbush,40.641500,-73.9594
7,Flushing,40.767500,-73.8331
8,Hunts Point/Longwood/Melrose,40.819567,-73.8941
9,Jamaica,40.702700,-73.7890


## Merge measles and locations data

In [9]:
locations_df.shape

(15, 3)

In [10]:
measles_df.shape

(16, 4)

In [11]:
df = pd.merge(measles_df, locations_df, on='neighborhood')

df

,neighborhood,ongoing_transmission,all_cases,new_cases,latitude,longitude
0,Borough Park,1,85,3,40.635000,-73.9921
1,Sunset Park,1,12,0,40.652700,-74.0093
2,Williamsburg,1,424,5,40.708100,-73.9571
3,Bensonhurst,0,2,0,40.613900,-73.9922
4,Brighton Beach/Coney Island,0,4,0,40.576800,-73.9652
5,Chelsea/Clinton,0,1,1,40.755150,-73.9966
6,Crown Heights,0,3,0,40.669400,-73.9422
7,Far Rockaway,0,1,0,40.599900,-73.7448
8,Flatbush,0,1,0,40.641500,-73.9594
9,Flushing,0,3,0,40.767500,-73.8331


In [12]:
df.shape

(15, 6)

## Add columns with color, radius, and label coordinates 

In [14]:
# Add color column

active_color   = 'red'  # '#ff4d4d' '#ff0000' '#8b0000'
inactive_color = 'blue' # '#4d4dff' '#0000ff' '#00008b'

df['ongoing_transmission_color'] = list(map(
    lambda x: active_color if x == 1 else inactive_color, 
    df['ongoing_transmission']
))

# Add radius columns

def cases_radius(x):
    if x == 0:
        return 0
    elif x < 10:
        return 10
    elif x < 100:
        return 20
    else:
        return 30
    
df['all_cases_radius'] = list(map(cases_radius, df['all_cases']))

df['new_cases_radius'] = list(map(cases_radius, df['new_cases']))

# Add label coordinates columns

df['label_latitude'] = df['latitude']

df['label_longitude'] = df['longitude']

df

,neighborhood,ongoing_transmission,all_cases,new_cases,latitude,longitude,ongoing_transmission_color,all_cases_radius,new_cases_radius,label_latitude,label_longitude
0,Borough Park,1,85,3,40.635000,-73.9921,red,20,10,40.635000,-73.9921
1,Sunset Park,1,12,0,40.652700,-74.0093,red,20,0,40.652700,-74.0093
2,Williamsburg,1,424,5,40.708100,-73.9571,red,30,10,40.708100,-73.9571
3,Bensonhurst,0,2,0,40.613900,-73.9922,blue,10,0,40.613900,-73.9922
4,Brighton Beach/Coney Island,0,4,0,40.576800,-73.9652,blue,10,0,40.576800,-73.9652
5,Chelsea/Clinton,0,1,1,40.755150,-73.9966,blue,10,10,40.755150,-73.9966
6,Crown Heights,0,3,0,40.669400,-73.9422,blue,10,0,40.669400,-73.9422
7,Far Rockaway,0,1,0,40.599900,-73.7448,blue,10,0,40.599900,-73.7448
8,Flatbush,0,1,0,40.641500,-73.9594,blue,10,0,40.641500,-73.9594
9,Flushing,0,3,0,40.767500,-73.8331,blue,10,0,40.767500,-73.8331


In [15]:
df.shape

(15, 11)

## Adjust the label coordinates manually

In [16]:
df['neighborhood']

0                     Borough Park
1                      Sunset Park
2                     Williamsburg
3                      Bensonhurst
4      Brighton Beach/Coney Island
5                  Chelsea/Clinton
6                    Crown Heights
7                     Far Rockaway
8                         Flatbush
9                         Flushing
10    Hunts Point/Longwood/Melrose
11                         Jamaica
12             Midwood/Marine Park
13                   Port Richmond
14                     Willowbrook
Name: neighborhood, dtype: object

In [17]:
#n = 'Willowbrook'

df.loc[df['neighborhood'] == 'Bensonhurst',                  ['label_latitude', 'label_longitude']] = [40.6,   -74.03]
df.loc[df['neighborhood'] == 'Borough Park',                 ['label_latitude', 'label_longitude']] = [40.63,  -74.087]
df.loc[df['neighborhood'] == 'Brighton Beach/Coney Island',  ['label_latitude', 'label_longitude']] = [40.565, -74.03]
df.loc[df['neighborhood'] == 'Chelsea/Clinton',              ['label_latitude', 'label_longitude']] = [40.765, -74.02]
df.loc[df['neighborhood'] == 'Crown Heights',                ['label_latitude', 'label_longitude']] = [40.669, -73.927]
df.loc[df['neighborhood'] == 'Far Rockaway',                 ['label_latitude', 'label_longitude']] = [40.585, -73.81]
df.loc[df['neighborhood'] == 'Flatbush',                     ['label_latitude', 'label_longitude']] = [40.641, -73.945]
df.loc[df['neighborhood'] == 'Flushing',                     ['label_latitude', 'label_longitude']] = [40.755, -73.85]
df.loc[df['neighborhood'] == 'Hunts Point/Longwood/Melrose', ['label_latitude', 'label_longitude']] = [40.807, -73.97]
df.loc[df['neighborhood'] == 'Jamaica',                      ['label_latitude', 'label_longitude']] = [40.69,  -73.81]
df.loc[df['neighborhood'] == 'Midwood/Marine Park',          ['label_latitude', 'label_longitude']] = [40.615, -73.93]
df.loc[df['neighborhood'] == 'Port Richmond',                ['label_latitude', 'label_longitude']] = [40.648, -74.15]
df.loc[df['neighborhood'] == 'Sunset Park',                  ['label_latitude', 'label_longitude']] = [40.67,  -74.05]
df.loc[df['neighborhood'] == 'Williamsburg',                 ['label_latitude', 'label_longitude']] = [40.708, -73.93]
df.loc[df['neighborhood'] == 'Willowbrook',                  ['label_latitude', 'label_longitude']] = [40.59,  -74.15]

#print(df.loc[df['neighborhood'] == n, ['latitude', 'longitude']])

#df.loc[df['neighborhood'] == n, ['label_latitude', 'label_longitude']]

df

,neighborhood,ongoing_transmission,all_cases,new_cases,latitude,longitude,ongoing_transmission_color,all_cases_radius,new_cases_radius,label_latitude,label_longitude
0,Borough Park,1,85,3,40.635000,-73.9921,red,20,10,40.630,-74.087
1,Sunset Park,1,12,0,40.652700,-74.0093,red,20,0,40.670,-74.050
2,Williamsburg,1,424,5,40.708100,-73.9571,red,30,10,40.708,-73.930
3,Bensonhurst,0,2,0,40.613900,-73.9922,blue,10,0,40.600,-74.030
4,Brighton Beach/Coney Island,0,4,0,40.576800,-73.9652,blue,10,0,40.565,-74.030
5,Chelsea/Clinton,0,1,1,40.755150,-73.9966,blue,10,10,40.765,-74.020
6,Crown Heights,0,3,0,40.669400,-73.9422,blue,10,0,40.669,-73.927
7,Far Rockaway,0,1,0,40.599900,-73.7448,blue,10,0,40.585,-73.810
8,Flatbush,0,1,0,40.641500,-73.9594,blue,10,0,40.641,-73.945
9,Flushing,0,3,0,40.767500,-73.8331,blue,10,0,40.755,-73.850


## Define function to create the maps

In [29]:
def cases_map_function(data, all_or_new):
    
    if all_or_new == 'all':
        title_prefix     = '<b>All</b>'
        cases_total      = all_cases_total
        start_date_label = all_start_date_label
        cases_col        = 'all_cases'
        radius_col       = 'all_cases_radius'
    elif all_or_new == 'new':
        title_prefix     = '<b>Newest</b>'
        cases_total      = new_cases_total
        start_date_label = new_start_date_label
        cases_col        = 'new_cases'
        radius_col       = 'new_cases_radius'
    else:
        return 'ERROR: invalid all_or_new argument (it should be "all" or "new").'
    
    # check if the start and end date labels have the same year
    if start_date_label[-6:] == end_date_label[-6:]:
        # if True, remove the year part from start date label 
        start_date_label = start_date_label[:-6]
    else:
        # if False, add a comma to the end of the start date label
        start_date_label = start_date_label + ','
    
    # Start figure
    fig = folium.Figure()
    
    # Figure title
    fig.html.add_child(folium.Element('<h3>' + title_prefix + ' NYC Measles Cases by Neighborhood</h3>'))
    
    # Figure sub-title
    fig.html.add_child(folium.Element('<h4><b>' + str(cases_total) + ' total cases from ' + start_date_label + ' to ' + end_date_label + '</b></h4>'))
    
    # Note about data and image sources
    fig.html.add_child(folium.Element('<h5 style="font-family:consolas;">Data: NYC Health, Image: carlos-afonso.github.io/measles</h5>'))
    
    # Base map
    cases_map = folium.Map(
        location   = [40.69, -73.945], 
        # tiles: OpenStreetMap, Mapbox Bright, Mapbox Control Room, Stamen (Terrain, Toner, and Watercolor)
        tiles      = 'Mapbox Bright', 
        height     = 540, 
        width      = 620, 
        zoom_start = 11, 
        minZoom = 11, 
        maxZoom = 11, 
        dragging = False, 
        doubleClickZoom = False, 
        scrollWheelZoom = False, 
        touchZoom = False, 
        zoom_control = False
    )
    
    for index, row in df[df[cases_col] > 0].iterrows():
        # Draw circles
        folium.CircleMarker(
            location = [row['latitude'], row['longitude']], 
            radius = row[radius_col], 
            #tooltip = row['neighborhood'] + ' (' + str(row[cases_col]) + ')', 
            color = row['ongoing_transmission_color'], 
            fill_color = row['ongoing_transmission_color']
        ).add_to(cases_map)
        # Write labels
        folium.Marker(
            location = [row['label_latitude'], row['label_longitude']], 
            icon = folium.DivIcon(html = (
                '<svg>'
                '<text x="0" y="10" fill=' + row['ongoing_transmission_color'] + ' font-size="14" font-weight="bold">' + 
                    row['neighborhood'].replace('/', ' / ') + ' (' + str(row[cases_col]) + ')</text>'
                '</svg>'
            ))
        ).add_to(cases_map)
    
    # Create "Size Legend"
    
    # Legend box (rectangle)
    folium.Marker(
        location = [40.825, -74.15], 
        icon = folium.DivIcon(html = (
            '<svg  width="180" height="170">'
            '<rect width="180" height="170" fill="white" fill-opacity="0.0" stroke="black" stroke-width="2"/>'
            '</svg>'
        ))
    ).add_to(cases_map)
    
    # Legend title
    folium.Marker(
        location = [40.822, -74.12], 
        icon = folium.DivIcon(html = (
            '<svg><text x="0" y="10" fill="black" font-size="14" font-weight="bold">Size Legend</text></svg>'
        ))
    ).add_to(cases_map)
    
    # Create circles
    folium.CircleMarker(
        location = [40.81, -74.13], radius = 10, color = 'black', fill_color = 'black'
    ).add_to(cases_map)
    folium.CircleMarker(
        location = [40.79, -74.13], radius = 20, color = 'black', fill_color = 'black'
    ).add_to(cases_map)
    folium.CircleMarker(
        location = [40.76, -74.13], radius = 30, color = 'black', fill_color = 'black'
    ).add_to(cases_map)
    
    # Add labels in front of circles
    folium.Marker(
        location = [40.81, -74.1], 
        icon = folium.DivIcon(html = (
            '<svg><text x="0" y="10" fill="black" font-size="12" font-weight="bold">1 to 9 cases</text></svg>'
        ))
    ).add_to(cases_map)
    folium.Marker(
        location = [40.79, -74.1], 
        icon = folium.DivIcon(html = (
            '<svg><text x="0" y="10" fill="black" font-size="12" font-weight="bold">10 to 99 cases</text></svg>'
        ))
    ).add_to(cases_map)
    folium.Marker(
        location = [40.76, -74.1], 
        icon = folium.DivIcon(html = (
            '<svg><text x="0" y="10" fill="black" font-size="12" font-weight="bold">100 or more cases</text></svg>'
        ))
    ).add_to(cases_map)
    
    # Create "Color Legend"
    
    # Legend box (rectangle)
    folium.Marker(
        location = [40.72, -74.15], 
        icon = folium.DivIcon(html = (
            '<svg  width="190" height="70">'
            '<rect width="190" height="70" fill="white" fill-opacity="0.0" stroke="black" stroke-width="2"/>'
            '</svg>'
        ))
    ).add_to(cases_map)
    
    # Legend title
    folium.Marker(
        location = [40.715, -74.12], 
        icon = folium.DivIcon(html = (
            '<svg><text x="0" y="10" fill="black" font-size="14" font-weight="bold">Color Legend</text></svg>'
        ))
    ).add_to(cases_map)
    
    # Legend labels
    folium.Marker(
        location = [40.705, -74.145], 
        icon = folium.DivIcon(html = (
            '<svg><text x="0" y="10" fill="red" font-size="12" font-weight="bold">With Ongoing Transmission</text></svg>'
        ))
    ).add_to(cases_map)
    folium.Marker(
        location = [40.695, -74.145], 
        icon = folium.DivIcon(html = (
            '<svg><text x="0" y="10" fill="blue" font-size="12" font-weight="bold">Without Ongoing Transmission</text></svg>'
        ))
    ).add_to(cases_map)

    fig.add_child(cases_map)
    
    return fig

## Create Total Cases Map

In [32]:
total_cases_map = cases_map_function(df, 'all')

total_cases_map.save('../images/nyc_measles_cases_by_neighborhood_map_all_py.html')

total_cases_map

## Create New Cases Map

In [33]:
new_cases_map = cases_map_function(df, 'new')

new_cases_map.save('../images/nyc_measles_cases_by_neighborhood_map_new_py.html')

new_cases_map

## Transform the Maps from HTML to PNG

At the moment this step is done "manually", as follows:
* In Firefox:
    * Open the HTML file/image in Firefox
    * (Optional: Increase the zoom in Firefox so that the image takes the most visible space possible)
    * Using the "Take a Screenshot" feature in Firefox, take a screenshot of the visible area, and download it as a PNG image.
* In GIMP:
    * Open the screenshot (PNG) image in GIMP
    * Auto-crop the image
    * Export the result as a PNG image

## Create All & New Cases Map (GIF)

In [35]:
images = [
    imageio.imread('../images/nyc_measles_cases_by_neighborhood_map_all_py.png'), 
    imageio.imread('../images/nyc_measles_cases_by_neighborhood_map_new_py.png'), 
]

imageio.mimsave('../images/nyc_measles_cases_by_neighborhood_map_all-new_py.gif', images, duration=5)